<a href="https://colab.research.google.com/github/HyunLee103/NER_korean/blob/yj/console_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Raw Data 로드

In [4]:
import os

BASE_DIR = '/content/drive/MyDrive/AI_Hackathon_konkuk2/baseline/data'

def load_data(file_name):
    with open(os.path.join(BASE_DIR, file_name), 'r', encoding='utf-8') as fp:
        return fp.readlines()

raw_train_data = load_data('ner_train.txt')
raw_test_data = load_data('ner_dev.txt')

In [5]:
tmp_data = raw_train_data[0]
tmp_data

'366\t새 벽 출 조 시 <SP> 야 영 적 극 <SP> 권 장 하 ㅂ 니 다 <SP> .\tB_TI I_TI O O O <SP> O O O O <SP> O O O O O O <SP> O\n'

## Feature 로드

```
{형태소: 161차원 vector}
```

In [6]:
import joblib

train_feature = joblib.load(os.path.join(BASE_DIR, 'train_concat_features(2)'))
test_feature = joblib.load(os.path.join(BASE_DIR, 'dev_concat_features(2)'))

train_feature['새벽'].shape

(184,)

## 통합 데이터셋 만들기

```
{형태소: 161차원 vector}
```

Input: 음절단위 문장

1. 합친다
2. 형태소로짼다
3. 아까했던 헝태소: 벡터 를 (길이) 만큼 한다
4. 그러면 (seq_len, 161) 의 벡터가 데이터 크기만큼 나온다.
5. <SOS\> 벡터, <EOS\> 벡터, <PAD\> 벡터를 준다.
6. ?

In [7]:
train_morph_token = joblib.load(os.path.join(BASE_DIR, 'train_morph_token'))
train_pos_token = joblib.load(os.path.join(BASE_DIR, 'train_pos_token'))

test_morph_token = joblib.load(os.path.join(BASE_DIR, 'test_morph_token'))
test_pos_token = joblib.load(os.path.join(BASE_DIR, 'test_pos_token'))

print(train_morph_token[0])
print(train_pos_token[0])
print(test_morph_token[0])
print(test_pos_token[0])

['새벽', '출조', '시', '<SP>', '야영', '적극', '<SP>', '권장', '하', 'ㅂ니다', '<SP>', '.']
['NNG', 'NNG', 'NNG', '<SP>', 'NNG', 'NNG', '<SP>', 'NNG', 'XSV', 'EF', '<SP>', 'SF']
['6', '일', '<SP>', '유통', '업계', '와', '<SP>', '정유', '업계', '에', '<SP>', '따르', '면', '<SP>', '‘', '이마트', '-', 'SK', '’', '<SP>', '간판', '을', '<SP>', '내걸', 'ㄴ', '<SP>', '주유소', '가', '<SP>', '올해', '<SP>', '안', '에', '<SP>', '등장', '하', 'ㄹ', '<SP>', '것', '이', '<SP>', '확실시', '되', '자', '<SP>', '이마트', '와', '<SP>', '경쟁', '관계', '에', '<SP>', '있', '는', '<SP>', '롯데마트', '<SP>', '홈플러스', '<SP>', '등', '<SP>', '다른', '<SP>', '대형', '<SP>', '마트', '도', '<SP>', '매장', '<SP>', '내', '<SP>', '주유소', '<SP>', '설립', '을', '<SP>', '위하', '아', '<SP>', '정유', '사와', '<SP>', '물밑', '에서', '<SP>', '활발', '하', '게', '<SP>', '접촉', '하', '고', '<SP>', '있', '다', '.']
['SN', 'NNBC', '<SP>', 'NNG', 'NNG', 'JC', '<SP>', 'NNG', 'NNG', 'JKB', '<SP>', 'VV', 'EC', '<SP>', 'SY', 'NNP', 'SY', 'SL', 'SY', '<SP>', 'NNG', 'JKO', '<SP>', 'VV+ETM', 'NNG', '<SP>', 'NNG', 'JKS', '<SP>', 'NNG', '<S

## `Dataset` 객체로 만들기

In [15]:
import os

def convert_sentence(sentence):
    return sentence.replace(' ', '').replace('<SP>', ' ')


def load_tag_dict():
    fp = open(os.path.join(BASE_DIR, 'tag_vocab.txt'), 'r')
    # tag_2_idx = {'<UNK>': 0, '<SP>': 1, '<EOS>': 2, '<PAD>': 3}
    # idx_2_tag = {0: '<UNK>', 1: '<SP>', 2: '<EOS>', 3: '<PAD>'}
    tag_2_idx = {'<PAD>': 0, '<UNK>': 1, '<SP>': 2, '<B_DT>':3, '<B_LC>':4, '<B_OG>':5,
                 '<B_PS>': 6, '<B_TI>': 7, '<I_DT>':8, '<I_LC>':9, '<I_OG>':10, '<I_PS>':11, '<I_TI>':12,
                 '<O>':13}
    idx_2_tag = {0:'<PAD>', 1: '<UNK>', 2: '<SP>', 3: '<B_DT>', 4: '<B_LC>', 5: '<B_OG>',
                 6: '<B_PS>', 7:'<B_TI>',8: '<I_DT>', 9: '<I_LC>',10: '<I_OG>',11: '<I_PS>',12: '<I_TI>',
                 13: '<O>'}

    index = 2
    for line in tqdm(fp.readlines()):
        tag = line.strip()
        tag_2_idx[tag] = index
        idx_2_tag[index] = tag
        index += 1

    return tag_2_idx, idx_2_tag

In [17]:
tag_2_idx, idx_2_tag = load_tag_dict()

100%|██████████| 12/12 [00:00<00:00, 12988.81it/s]


In [18]:
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset

PAD_TOKEN = 0
UNK_TOKEN = 1
SP_TOKEN = 2

FEATURE_SIZE = 184

UNK_VECTOR = [UNK_TOKEN] * FEATURE_SIZE
SP_VECTOR = [SP_TOKEN] * FEATURE_SIZE
PAD_VECTOR = [PAD_TOKEN] * FEATURE_SIZE

device = 'cuda' if torch.cuda.is_available() else 'cpu'


class NERDataset(Dataset):
    def __init__(self, raw_data, morphs_list, pos_list, feature=None, device='cpu'):
        super(NERDataset, self).__init__()
        self.source_list = []
        self.target_list = []

        tag_dict = tag_2_idx
        max_length = self._get_max_length(raw_data)

        for row, morphs, pos in tqdm(zip(raw_data, morphs_list, pos_list)):
            index, syllables, tags = row.rstrip('\n').split('\t')
            syllables_list = syllables.split()
            tags_list = tags.split()

            encoded_syllable_list = []
            if feature:
                for morph in morphs:
                    if morph in feature.keys():
                        morph_size = len(morph) if morph != '<SP>' else 1
                        encoded_syllable_list += [feature[morph]] * morph_size
                    else:
                        encoded_syllable_list += [UNK_VECTOR] * len(morph)

                padding_size = max_length - len(encoded_syllable_list)
                encoded_syllable_list += [PAD_VECTOR] * padding_size

                self.source_list.append(encoded_syllable_list)
            else:
                # TODO: raw feature generation
                pass

            encoded_tag_list = []
            for tag in tags_list:
                if tag in tag_dict.keys():
                    encoded_tag_list.append(tag_dict[tag])
                else:
                    encoded_tag_list.append(tag_dict['<UNK>'])

            padding_size = max_length - len(encoded_tag_list)
            encoded_tag_list += [tag_dict['<PAD>']] * padding_size

            self.target_list.append(encoded_tag_list)

        self.source = torch.tensor(self.source_list).to(device)
        self.target = torch.tensor(self.target_list).to(device)

    def _get_max_length(self, raw_data):
        max_length = 0
        for row in raw_data:
            index, syllables, tags = row.rstrip('\n').split('\t')
            syllables_list = syllables.split()
            length = len(syllables_list)
            if max_length < length:
                max_length = length
        return max_length

    def __str__(self):
        return 'source: {}, target: {}'.format(self.source.shape, self.target.shape)

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return {
            'source': self.source[idx],
            'target': self.target[idx],
        }


print()

train_dataset = NERDataset(raw_train_data, train_morph_token, train_pos_token, train_feature, device=device)
test_dataset = NERDataset(raw_test_data, test_morph_token, test_pos_token, test_feature, device=device)

print(train_dataset)
print(test_dataset)


1340it [00:00, 13391.39it/s]

7319it [00:00, 12277.69it/s]
995it [00:00, 16375.45it/s]


source: torch.Size([7319, 491, 184]), target: torch.Size([7319, 491])
source: torch.Size([995, 220, 184]), target: torch.Size([995, 220])


In [ ]:
# joblib.dump(train_dataset, 'train_dataset')
# joblib.dump(test_dataset, 'test_dataset')

In [19]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print('total steps: {}'.format(len(train_loader)), end='\n\n')

sample = next(iter(train_loader))
print('shape of {}: {}'.format('source', sample['source'].shape))  # ([64, 492, 184]) -> 491
print('shape of {}: {}'.format('target', sample['target'].shape), end='\n\n')  # ([64, 492]) -> 491

total steps: 115

shape of source: torch.Size([64, 491, 184])
shape of target: torch.Size([64, 491])



In [20]:
test_dataset = NERDataset(raw_test_data, test_morph_token, test_pos_token, test_feature, device=device)

995it [00:00, 16535.45it/s]


In [21]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print('total steps: {}'.format(len(test_loader)), end='\n\n')
sample = next(iter(train_loader))

total steps: 995



## 모델링

In [22]:
!pip install pytorch-crf

In [23]:
!pip install seqeval

     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=19aa4e99634addf9cbbbfd46a48495fd513aa8d09b28a436ce6370df50b55b66
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [35]:
!pip install glove_python
from glove import Corpus, Glove

     |████████████████████████████████| 266kB 6.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700301 sha256=ee63495c1d6ce5ca0720534d417fd0e8126fb603a7ca2e1f08468400e4abb2ad
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [38]:
# load glove
glove_model = Glove.load('/content/drive/MyDrive/AI_Hackathon_konkuk2/baseline/glove_total.model')
print(f'Load glove_model...{str(glove_model)}')

# total word dict
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('Lengh of word dict... : ', len(word_dict))

Load glove_model...<glove.glove.Glove object at 0x7fd4c99942b0>
Lengh of word dict... :  23583


In [66]:
import torch
import torch.nn as nn
from torchcrf import CRF

matrix_len = 23583
weights_matrix = np.zeros((matrix_len, 128))
words_found = 0

for i, word in enumerate(word_dict.keys()):
    try: 
        weights_matrix[i] = word_dict[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

print(weights_matrix)

def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict(word_dict)
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class ToyNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super(self).__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))


[[-0.26275949 -0.09441504 -0.08714107 ...  0.1316752   0.11281312
   0.05665073]
 [ 0.05791898 -0.2535104   0.01162866 ... -0.00509863 -0.00329273
  -0.01341859]
 [-0.35233491 -0.86255513 -0.15164623 ...  0.09656302  0.16354855
   0.11602079]
 ...
 [ 0.0201278   0.00086433  0.03706049 ... -0.02742726 -0.03753794
  -0.02662013]
 [ 0.06951162 -0.05345564  0.04598201 ... -0.04409245 -0.04996535
  -0.04715683]
 [ 0.05358822 -0.03530177  0.04555802 ... -0.04042755 -0.05540955
  -0.04958085]]


In [67]:
import torch
import torch.nn as nn
from torchcrf import CRF

matrix_len = 23583
weights_matrix = np.zeros((matrix_len, 128))
words_found = 0

for i, word in enumerate(word_dict.keys()):
    try: 
        weights_matrix[i] = word_dict[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

print(weights_matrix)

def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim


class RNN_CRF(nn.Module):
    def __init__(self, pretrained_weight, embedding_size, hidden_size, output_size, dropout=0.5):
        super(RNN_CRF, self).__init__()

        # self.embedding = nn.Embedding.from_pretrained(pretrained_weight)
        self.embedding,_, _ = create_emb_layer(pretrained_weight, True)
        
        self.dropout = nn.Dropout(dropout)

        self.rnn = nn.GRU(
            embedding_size,
            hidden_size,
            batch_first=True,
            bidirectional=True
        )

        # CRF layer
        self.crf = CRF(output_size, batch_first=True)

        # (batch_size, seq_len, hidden_size * 2) -> (batch_size, seq_len, output_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, inputs, labels=None):
        # (batch_size, seq_len) -> (batch_size, seq_len, embedding_size)
        inputs = inputs.float()
        outputs, hidden = self.rnn(inputs)

        # (batch_size, seq_len, hidden_size * 2)
        outputs = self.dropout(outputs)

        # (batch_size, seq_len, hidden_size * 2) -> (batch_size, seq_len, output_size)
        logits = self.fc(outputs)

        if labels is not None:
            log_likelihood = self.crf(
                emissions=logits,
                tags=labels,
                reduction="mean"
            )
            loss = log_likelihood * -1.0
            return loss
        else:
            output = self.crf.decode(emissions=logits)
            return output

[[-0.26275949 -0.09441504 -0.08714107 ...  0.1316752   0.11281312
   0.05665073]
 [ 0.05791898 -0.2535104   0.01162866 ... -0.00509863 -0.00329273
  -0.01341859]
 [-0.35233491 -0.86255513 -0.15164623 ...  0.09656302  0.16354855
   0.11602079]
 ...
 [ 0.0201278   0.00086433  0.03706049 ... -0.02742726 -0.03753794
  -0.02662013]
 [ 0.06951162 -0.05345564  0.04598201 ... -0.04409245 -0.04996535
  -0.04715683]
 [ 0.05358822 -0.03530177  0.04555802 ... -0.04042755 -0.05540955
  -0.04958085]]


## Train and Evaluate

In [50]:
import torch
import torch.optim as optim
from seqeval.metrics import classification_report


def train(model, train_data, test_data=None, num_epochs=20):
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    accuracy_list = []

    for epoch in range(num_epochs):
        model.train()
        losses = []

        for step, batch in enumerate(train_data):
            source = batch['source']
            target = batch['target']
            
            loss = model.forward(source, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step + 1) % 50 == 0:
                print('{} step processed.. current loss : {}'.format(step + 1, loss.data.item()))
            losses.append(loss.data.item())

        print('Average Loss : {}'.format(np.mean(losses)))

        torch.save(model, 'savepoint.model')
        do_test(model, test_data)


def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()


def do_test(model, test_dataloader):
    model.eval()
    
    _, idx2tag = load_tag_dict()

    predicts, answers = [], []
    
    for step, batch in enumerate(test_dataloader):
        source = batch['source']
        target = batch['target']

        # 예측 라벨 출력
        output = model(source)

        # 성능 평가를 위해 예측 값과 정답 값 리스트에 저장
        for idx, answer in enumerate(tensor2list(target)):
            answers.append([idx2tag[e].replace("_", "-") for e in answer if idx2tag[e] != "<SP>" and idx2tag[e] != "<PAD>"])
            predicts.append([idx2tag[e].replace("_", "-") for i, e in enumerate(output[idx]) if idx2tag[answer[i]] != "<SP>" and idx2tag[answer[i]] != "<PAD>"] )

    print(len(predicts))
    
    # 성능 평가
    print(classification_report(answers, predicts))
    
    print(answers)
    print(predicts)

In [56]:
# load glove embeddings
import os
import joblib
import torch

train_weights = joblib.load(os.path.join(BASE_DIR, 'train_emb_word_dict_mecab_sp.pickle'))
dev_weights = joblib.load(os.path.join(BASE_DIR, 'dev_emb_word_dict_mecab_sp.pickle'))

weights = torch.FloatTensor(list({**train_weights, **dev_weights}.values()))#.cuda()

print(weights.shape)

torch.Size([23583, 128])


In [ ]:
input_size = 184
hidden_size = 128
output_size = 14
dropout = 0.3

model = RNN_CRF(weights, input_size, hidden_size, output_size, dropout=dropout).to(device)
print(model)

train(model, train_loader, test_loader)

RNN_CRF(
  (embedding): Embedding(23583, 128)
  (dropout): Dropout(p=0.3, inplace=False)
  (rnn): GRU(184, 128, batch_first=True, bidirectional=True)
  (crf): CRF(num_tags=14)
  (fc): Linear(in_features=256, out_features=14, bias=True)
)
50 step processed.. current loss : 18.92542266845703
100 step processed.. current loss : 12.07421875


100%|██████████| 12/12 [00:00<00:00, 53148.52it/s]

Average Loss : 44.04367164943529


995


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <PAD> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

100%|██████████| 12/12 [00:00<00:00, 39290.90it/s]

Average Loss : 8.164959339473558


995
              precision    recall  f1-score   support

          DT       0.59      0.35      0.44       624
          LC       0.19      0.08      0.11       537
          OG       0.26      0.19      0.22       973
        PAD>       0.00      0.00      0.00         0
          PS       0.32      0.12      0.17       742
          TI       0.15      0.06      0.09        95

   micro avg       0.33      0.18      0.24      2971
   macro avg       0.25      0.13      0.17      2971
weighted avg       0.33      0.18      0.23      2971

[['B-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'O', 

100%|██████████| 12/12 [00:00<00:00, 30633.99it/s]

Average Loss : 7.013598877450694


## Test Macro F1

In [ ]:
do_test(model, test_loader)

100%|██████████| 12/12 [00:00<00:00, 29127.11it/s]


995


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          DT       0.75      1.00      0.85    154027
          LC       0.00      0.00      0.00       537
          OG       0.00      0.00      0.00       973
          PS       0.00      0.00      0.00       742
          TI       0.00      0.00      0.00        95

   micro avg       0.75      0.98      0.85    156374
   macro avg       0.15      0.20      0.17    156374
weighted avg       0.74      0.98      0.84    156374

